In [151]:
from selenium.webdriver.common.by import By
import selenium as sel
import pandas as pd
import time
from datetime import datetime
import base as bs

import os as os
import csv

from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException


In [152]:
meses = {"Enero": "1","Febrero": "2", "Marzo": "3", "Abril": "4", "Mayo": "5", "Junio": "6", "Julio": "7", 
        "Agosto": "8", "Septiembre": "9", "Octubre": "10", "Noviembre": "11", "Diciembre": "12"}

In [153]:
def get_url(art):
    try:
        url = art.find_element(By.XPATH, './/h2[@class="p boton extra-bold mb-10"]//a').get_attribute('href')
    except:
        url = None
    else:
        return url


In [154]:
def get_titulo(art):
    try:
        titulo = art.find_element(By.XPATH, './/h2[@class="p boton extra-bold mb-10"]//a').text
    except:
        titulo = None
    else:
        return titulo

In [155]:
def get_autor(art):
    try:
        autor = art.find_element(By.XPATH, './/div[@class="mainInternalArticle__autor mb-10"]//a').text
    except:
        autor = None
    else:
        return autor

In [156]:
def get_fecha(art):
    try:
        fecha = art.find_element(By.XPATH, './/time').text
        lst_fecha = fecha.split()
        dia = lst_fecha[1][:-1]
        mes = meses[lst_fecha[0]]
        ano = lst_fecha[2]
        
        fecha_pub = datetime.strptime(dia+"/"+mes+"/"+ano, "%d/%m/%Y")
    except:
        fecha_pub = None
    return fecha_pub

In [157]:
def get_resumen(art):
    try:
        resumen = art.find_element(By.XPATH, './/h2').text
    except:
        resumen = None
    return resumen

In [158]:
def get_imag(art):
    try:
        imagen = art.find_element(By.XPATH, './/div[contains(@class, "mainHistoria-imagen")]//img').get_attribute('src')
    except:
        imagen = None
    return imagen

In [159]:
def get_contenido(driver):
    try:
        parrafos = driver.find_elements(By.XPATH, './/div[contains(@class, "mainHistoria-interna mainHistoria-description")]//p')
        contenido = '\n\n'.join(list(map(lambda x: x.text, parrafos)))
    except:
        contenido = None
    return contenido

In [160]:
#Empresa con la cual vamos a extraer los articulos
empresa = input("Digite la empresa a extraer: ").lower()
revista = "laSillaVacia"

In [161]:
# crear driver... MODIFICAR DEPENDIENDO DEL NAVEGADOR
driver = sel.webdriver.Edge()
driver.get(f'https://www.lasillavacia.com/buscar?q={empresa}&cat=all')
time.sleep(2)

i = 1
try:
    while i < 2:
        button = driver.find_element(By.XPATH, './/a[contains(@class, "load-more-results d-block p xl black-3d bold uppercase text-center")]')
        button.click()
        time.sleep(4)
except:
    pass

In [162]:
# "c c-d _g _g-md c-m-l c--m-n" Contiene tema
articulos = driver.find_elements(By.XPATH, './/article')

In [163]:
titulares = []
for art in articulos:
    url = get_url(art)
    if url == None:
        continue
    # if not(bs.existedb(url, revista)):
    titulo = get_titulo(art)
    fecha = get_fecha(art)
    titulares.append({'Fecha Extraccion':datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
                    'Titulo': titulo,
                    'Fecha Publicacion':fecha,
                    'URL': url,
                    'Empresa': empresa,
                    })

In [164]:
for tit in titulares:
    driver.get(tit["URL"])
    tit["Autor"] = get_autor(driver)
    tit["Imagen"] = get_imag(driver)
    # tit["Resumen"] = get_resumen(driver)
    # tit["Fuente"] = "La Silla Vacía"
    tit["Contenido"] = get_contenido(driver)

In [165]:
"""
['Fecha Extraccion', 'Titulo', 'Fecha Publicacion', 
'URL', 'Imagen', 'Empresa', 'Autor', 'Contenido']
"""
columnas = list(titulares[0].keys())

IndexError: list index out of range

In [ ]:
# Para AGREGAR titulares a un archivo.
# with open(f'../data/raw/{revista}.csv', 'a', newline='', errors='ignore') as csv_file:
#     dict_object = csv.DictWriter(csv_file, fieldnames=columnas) 
  
#     dict_object.writerows(titulares)

In [ ]:
#Para crear un NUEVO archivo con los titulares
pd.json_normalize(titulares).to_csv(f'../data/raw/{revista}.csv',index=False, encoding='latin-1', errors='ignore', columns=columnas)

In [ ]:
# Demora inteligente
# delay = 10
# try:
#     url = WebDriverWait(driver, delay).until(EC.presence_of_element_located(By.XPATH, ".//div"))
# except:
#     print("La página tardó demasiado en cargar")

In [ ]:
aux1 = "Diciembre 13, 2021".split()
aux1

['Diciembre', '13,', '2021']

In [ ]:
aux1[1] = aux1[1][:-1]
aux1

['Diciembre', '13', '2021']